In [11]:
import xml.etree.ElementTree as ET
import mysql.connector
import logging
import uuid


In [93]:
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="AmeSame420",
        database="mydb"
    )


ns = {'ns': 'http://www.battlescribe.net/schema/catalogueSchema'}


def insert_faction(root, cursor):
    _name = root.get('name')
    _description = root.get('description')
    if _description is None:
        _description = ""
    _id = root.get('id')

    sql = "INSERT INTO Faction (name, description, id) VALUES (%s, %s,%s)"
    try:
        cursor.execute(sql, (_name, _description, _id))
    except Exception as e:
        logging.error("Error at %s", "insert_faction", exc_info=e)
        print(e)

    return _id


def insert_unit(cursor, root, faction_id):
    categoryEntries = root.find('ns:categoryEntries', ns)
    for categoryEntry in categoryEntries:
        if "[Legends]" not in categoryEntry.get("name"):
            _name = categoryEntry.get("name")
            _description = categoryEntry.get("description")
            if _description is None:
                _description = ""
            _id = categoryEntry.get("id")
            sql = "INSERT INTO Unit (name, description, id, faction_id) VALUES (%s, %s,%s,%s)"
            try:
                cursor.execute(sql, (_name, _description, _id, faction_id))
                parse_selectionEntry(cursor, categoryEntry, _id)
            except Exception as e:
                logging.error("Error at %s", "insert_unit", exc_info=e)
                print(e)


def insert_unit2(cursor, root, faction_id):
    sharedSelectionEntries = root.find('ns:sharedSelectionEntries', ns)
    for selectionEntry in sharedSelectionEntries:
        if "[Legends]" not in selectionEntry.get("name"):
            if selectionEntry.get("type") == "model" or selectionEntry.get("type") == "unit":

                _name = selectionEntry.get("name")
                _description = selectionEntry.get("description")
                _description = ""
                _id = selectionEntry.get("id")

                sql = "INSERT INTO Unit (name, description, id, faction_id) VALUES (%s, %s,%s,%s)"
                try:
                    cursor.execute(sql, (_name, _description, _id, faction_id))
                    parse_selectionEntry(cursor, selectionEntry, _id)
                    insert_keywords(cursor, selectionEntry, _id)
                except Exception as e:
                    logging.error("Error at %s", "insert_unit", exc_info=e)
                    print(e)

def insert_keywords(cursor, selectionEntry, unit_id):
    keywords = selectionEntry.findall(".//ns:categoryLink", ns)
    for keyword in keywords:
        _name = keyword.get("name")
        _id = keyword.get("targetId")
        sql = "INSERT IGNORE INTO ModelKeyWord (text, id) VALUES (%s, %s)"
        sql_map = "INSERT IGNORE INTO unit_has_ModelKeyword (Unit_id, ModelKeyword_id) VALUES (%s, %s)"
        try:
            cursor.execute(sql, (_name, _id))
            cursor.execute(sql_map, (unit_id, _id))
        except Exception as e:
            logging.error("Error at %s", "insert_keywords", exc_info=e)
            print(e)

def parse_selectionEntry(cursor, selectionEntry, unit_id):
    profiles = selectionEntry.findall(".//ns:profile", ns)
    for profile in profiles:
        if profile.get("typeName") == "Unit" or profile.get("typeName") == "Model":
            insert_model(cursor, profile, unit_id)
        if profile.get("typeName") == "Abilities":
            insert_abilities(cursor, profile, unit_id)
            print("inserting abilities")



def insert_abilities(cursor, profile, unit_id):
    _name = profile.get("name")
    _id = profile.get("id")
    _description = profile.findall(".//ns:characteristic", ns)[0].text
    print(_name)
    print(_id)
    print(_description)

    sql = "INSERT INTO modelability (name, description, id) VALUES (%s, %s,%s)"
    sql_map = "INSERT INTO unit_has_modelability (unit_id, ModelAbility_id) VALUES (%s, %s)"
    try:
        cursor.execute(sql, (_name, _description, _id))
        cursor.execute(sql_map, (unit_id, _id))
    except Exception as e:
        logging.error("Error at %s", "insert_abilities", exc_info=e)
        print(e)

def create_keywords_sql(cursor, keywords, _id):
    _id = str(uuid.uuid4())
    for keyword in keywords:
        # Check if the keyword already exists
        cursor.execute("SELECT id FROM weaponkeyword WHERE text = %s", (keyword,))
        result = cursor.fetchone()

        # If the keyword does not exist, insert it
        if result is None:
            cursor.execute("INSERT INTO weaponkeyword (id, text) VALUES (%s, %s)", (_id, keyword))


def insert_model(cursor, model, unit_id):
    _name = model.get("name")
    _id = model.get("id")

    move = None
    leadership = None
    save = None
    toughness = None
    wounds = None
    oc = None
    keywords = []
    for characteristics in model.findall(".//ns:characteristic", ns):

        if characteristics.get("name") == "M":
            move = characteristics.text
        elif characteristics.get("name") == "LD":
            leadership = characteristics.text
        elif characteristics.get("name") == "SV":
            save = characteristics.text
        elif characteristics.get("name") == "T":
            toughness = characteristics.text
        elif characteristics.get("name") == "W":
            wounds = characteristics.text
        elif characteristics.get("name") == "OC":
            oc = characteristics.text


        sql_models = "INSERT INTO Model (id, name, move, toughness,wounds, leadership, save, unit_id, oc) VALUES (%s, %s,%s,%s, %s, %s,%s,%s,%s)"


    try:
        cursor.execute(sql_models, (_id, _name, move, toughness, wounds, leadership, save, unit_id, oc))

    except Exception as e:
        logging.error("Error at %s", "insert_model", exc_info=e)
        print(e)


def parse_cat_file(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    return root


def ingest_data(root, cursor):
    faction_id = insert_faction(root, cursor)
    insert_unit2(cursor, root, faction_id)
    db_connection.commit()


def clear_all_tables(cursor):
    # Get all table names
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()

    # Iterate over all tables and delete all records
    for table in tables:
        try:
            cursor.execute(f"DELETE FROM `{table[0]}`")
        except mysql.connector.Error as err:
            print(f"Something went wrong: {err}")



In [94]:
db_connection = connect_to_db()
cursor = db_connection.cursor()


In [95]:
root = parse_cat_file("./wh40k-10e/Imperium - Salamanders.cat")
clear_all_tables(cursor)
ingest_data(root, cursor)

Unto the Anvil
5a40-93b3-6c2a-f7cb
While this model is leading a unit, each time a model in that unit makes a melee attack, you can re-roll the Wound roll.
inserting abilities
Lord of the Pyroclasts
68e4-3da0-f6ab-8624
While an enemy unit is within Engagement Range of this model, halve the Objective Control characteristic of models in that enemy unit
inserting abilities
Leader
f678-23f8-a491-c7d2
This model can be attached to the following units:

■ Assault Intercessor Squad
■ Bladeguard Veteran Squad
■ Company Heroes
■ Infernus Squad
■ Intercessor Squad
■ Sternguard Veteran Squad
■ Tactical Squad
inserting abilities
Forgefather
b423-2ef4-b87c-b65d
In your Shooting phase, select one enemy unit within 24" of and visible to this model. Until the end of the phase, each time a friendly Adeptus Astartes model makes a ranged attack with a Torrent or Melta weapon that targets that enemy unit, you can re-roll the Wound roll
inserting abilities
Seeker of Lost Relics
22fa-126d-31e4-6a5a
At the s